# Collecting temperature data from an API

## About the data
In this notebook, we will be collecting daily temperature data from the [National Centers for Environmental Information (NCEI) API](https://www.ncdc.noaa.gov/cdo-web/webservices/v2). We will use the Global Historical Climatology Network - Daily (GHCND) dataset; see the documentation [here](https://www1.ncdc.noaa.gov/pub/data/cdo/documentation/GHCND_documentation.pdf).

*Note: The NCEI is part of the National Oceanic and Atmospheric Administration (NOAA) and, as you can see from the URL for the API, this resource was created when the NCEI was called the NCDC. Should the URL for this resource change in the future, you can search for "NCEI weather API" to find the updated one.*

## Using the NCEI API
Request your token [here](https://www.ncdc.noaa.gov/cdo-web/token) and then paste it below.

In [18]:
import requests

def make_request(endpoint, payload=None):
    """
    Make a request to a specific endpoint on the weather API
    passing headers and optional payload.
    
    Parameters:
        - endpoint: The endpoint of the API you want to 
                    make a GET request to.
        - payload: A dictionary of data to pass along 
                   with the request.
    
    Returns:
        A response object.
    """
    return requests.get(
        f'https://www.ncdc.noaa.gov/cdo-web/api/v2/{endpoint}',
        headers={
            'token': 'PASTE_TOKEN_HERE'
        },
        params=payload
    )

**Note: the API limits us to 5 requests per second and 10,000 requests per day.**

## See which datasets are available
We can make requests to the `datasets` endpoint to see which datasets are available. We also pass in a dictionary for the payload to get datasets that have data after the start date of October 1, 2018.

In [19]:
response = make_request('datasets', {'startdate': '2018-10-01'})

Status code of `200` means everything is OK. More codes can be found [here](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).

In [20]:
response.status_code

200

Alternatively, we can check the `ok` attribute:

In [21]:
response.ok

True

### Get the keys of the result
The result is a JSON payload, which we can access with the `json()` method of our response object. JSON objects can be treated like dictionaries, so we can access the keys just like we would a dictionary:

In [22]:
payload = response.json()
payload.keys()

dict_keys(['metadata', 'results'])

The metadata of the response will tell us information about the request and data we got back:

In [23]:
payload['metadata']

{'resultset': {'offset': 1, 'count': 11, 'limit': 25}}

### Figure out what data is in the result
The `results` key contains the data we requested. This is a list of what would be rows in our dataframe. Each entry in the list is a dictionary, so we can look at the keys to get the fields:

In [24]:
payload['results'][0].keys()

dict_keys(['uid', 'mindate', 'maxdate', 'name', 'datacoverage', 'id'])

### Parse the result
We don't want all those fields, so we will use a list comphrension to take only the `id` and `name` fields out:

In [25]:
list(
    (data['id'], data['name']) 
    for data in payload['results']
    )

[('GHCND', 'Daily Summaries'),
 ('GSOM', 'Global Summary of the Month'),
 ('GSOY', 'Global Summary of the Year'),
 ('NEXRAD2', 'Weather Radar (Level II)'),
 ('NEXRAD3', 'Weather Radar (Level III)'),
 ('NORMAL_ANN', 'Normals Annual/Seasonal'),
 ('NORMAL_DLY', 'Normals Daily'),
 ('NORMAL_HLY', 'Normals Hourly'),
 ('NORMAL_MLY', 'Normals Monthly'),
 ('PRECIP_15', 'Precipitation 15 Minute'),
 ('PRECIP_HLY', 'Precipitation Hourly')]

## Figure out which data category we want
The `GHCND` data containing daily summaries is what we want. Now we need to make another request to figure out which data categories we want to collect. This is the `datacategories` endpoint. We have to pass the `datasetid` for `GHCND` as the payload so the API knows which dataset we are asking about:

In [26]:
# get data category id

response = make_request(
    'datacategories', 
    payload={'datasetid': 'GHCND'}
    )
response.status_code

200

Since we know the API gives us a `metadata` and a `results` key in each response, we can see what is in the `results` portion of the JSON payload:

In [27]:
response.json()['results']

[{'name': 'Evaporation', 'id': 'EVAP'},
 {'name': 'Land', 'id': 'LAND'},
 {'name': 'Precipitation', 'id': 'PRCP'},
 {'name': 'Sky cover & clouds', 'id': 'SKY'},
 {'name': 'Sunshine', 'id': 'SUN'},
 {'name': 'Air Temperature', 'id': 'TEMP'},
 {'name': 'Water', 'id': 'WATER'},
 {'name': 'Wind', 'id': 'WIND'},
 {'name': 'Weather Type', 'id': 'WXTYPE'}]

## Grab the data type ID for the temperature category
We will be working with temperatures, so we want the `TEMP` data category. Now, we need to find the `datatypes` to collect. For this, we use the `datatypes` endpoint and provide the `datacategoryid` which was `TEMP`. We also specify a limit for the number of `datatypes` to return with the payload. If there are more than this we can make another request later, but for now, we just want to pick a few out:

In [28]:
# get data type id

response = make_request(
    'datatypes',
    payload={
        'datacategoryid': 'TEMP', 
        'limit': 100
        }
    )
response.status_code

200

We can grab the `id` and `name` fields for each of the entries in the `results` portion of the data. The fields we are interested in are at the bottom:

In [29]:
datatypes = list(
    (datatype['id'], datatype['name']) 
    for datatype in response.json()['results']
    )
datatypes[-5:] # look at the last 5

[('MNTM', 'Monthly mean temperature'),
 ('TAVG', 'Average Temperature.'),
 ('TMAX', 'Maximum temperature'),
 ('TMIN', 'Minimum temperature'),
 ('TOBS', 'Temperature at the time of observation')]

## Determine which location category we want
Now that we know which `datatypes` we will be collecting, we need to find the location to use. First, we need to figure out the location category. This is obtained from the `locationcategories` endpoint by passing the `datasetid`:

In [30]:
# get location category id 

response = make_request(
    'locationcategories', 
    payload={'datasetid': 'GHCND'}
)
response.status_code

200

We can use `pprint` to print dictionaries in an easier-to-read format. After doing so, we can see there are 12 different location categories, but we are only interested in `CITY`:

In [31]:
import pprint
pprint.pprint(response.json())

{'metadata': {'resultset': {'count': 12, 'limit': 25, 'offset': 1}},
 'results': [{'id': 'CITY', 'name': 'City'},
             {'id': 'CLIM_DIV', 'name': 'Climate Division'},
             {'id': 'CLIM_REG', 'name': 'Climate Region'},
             {'id': 'CNTRY', 'name': 'Country'},
             {'id': 'CNTY', 'name': 'County'},
             {'id': 'HYD_ACC', 'name': 'Hydrologic Accounting Unit'},
             {'id': 'HYD_CAT', 'name': 'Hydrologic Cataloging Unit'},
             {'id': 'HYD_REG', 'name': 'Hydrologic Region'},
             {'id': 'HYD_SUB', 'name': 'Hydrologic Subregion'},
             {'id': 'ST', 'name': 'State'},
             {'id': 'US_TERR', 'name': 'US Territory'},
             {'id': 'ZIP', 'name': 'Zip Code'}]}


## Get NYC Location ID
In order to find the location ID for New York, we need to search through all the cities available. Since we can ask the API to return the cities sorted, we can use binary search to find New York quickly without having to make many requests or request lots of data at once. The following function makes the first request to see how big the list is and looks at the first value. From there it decides if it needs to move towards the beginning or end of the list by comparing the item we are looking for to others alphabetically. Each time it makes a request it can rule out half of the remaining data to search.

In [32]:
def get_item(name, what, endpoint, start=1, end=None):
    """
    Grab the JSON payload for a given field by name using binary search.

    Parameters:
        - name: The item to look for.
        - what: Dictionary specifying what the item in `name` is.
        - endpoint: Where to look for the item.
        - start: The position to start at. We don't need to touch this, but the
                 function will manipulate this with recursion.
        - end: The last position of the items. Used to find the midpoint, but
               like `start` this is not something we need to worry about.

    Returns:
        Dictionary of the information for the item if found otherwise 
        an empty dictionary.
    """
    # find the midpoint which we use to cut the data in half each time
    mid = (start + (end or 1)) // 2
    
    # lowercase the name so this is not case-sensitive
    name = name.lower()
    
    # define the payload we will send with each request
    payload = {
        'datasetid': 'GHCND',
        'sortfield': 'name',
        'offset': mid, # we will change the offset each time
        'limit': 1 # we only want one value back
    }
    
    # make our request adding any additional filter parameters from `what`
    response = make_request(endpoint, {**payload, **what})
    
    if response.ok:
        payload = response.json()

        # if response is ok, grab the end index from the response metadata the first time through
        end = end or payload['metadata']['resultset']['count']
        
        # grab the lowercase version of the current name
        current_name = payload['results'][0]['name'].lower()
        
        # if what we are searching for is in the current name, we have found our item
        if name in current_name:
            return payload['results'][0] # return the found item
        else:
            if start >= end: 
                # if our start index is greater than or equal to our end, we couldn't find it
                return {}
            elif name < current_name:
                # our name comes before the current name in the alphabet, so we search further to the left
                return get_item(name, what, endpoint, start, mid - 1)
            elif name > current_name:
                # our name comes after the current name in the alphabet, so we search further to the right
                return get_item(name, what, endpoint, mid + 1, end)    
    else:
        # response wasn't ok, use code to determine why
        print(f'Response not OK, status: {response.status_code}')

When we use binary search to find New York, we find it in just 8 requests despite it being close to the middle of 1,983 entries:

In [33]:
# get NYC id 

nyc = get_item(
    'New York', 
    {'locationcategoryid': 'CITY'}, 
    'locations'
    )
nyc

{'mindate': '1869-01-01',
 'maxdate': '2021-08-28',
 'name': 'New York, NY US',
 'datacoverage': 1,
 'id': 'CITY:US360019'}

## Get the station ID for Central Park
The most granular data is found at the station level:

In [44]:
central_park = get_item(
    'NY City Central Park', 
    {'locationid': nyc['id']}, 'stations'
    )
central_park

{'elevation': 42.7,
 'mindate': '1869-01-01',
 'maxdate': '2021-08-27',
 'latitude': 40.77898,
 'name': 'NY CITY CENTRAL PARK, NY US',
 'datacoverage': 1,
 'id': 'GHCND:USW00094728',
 'elevationUnit': 'METERS',
 'longitude': -73.96925}

## Request the temperature data
Finally, we have everything we need to make our request for the New York temperature data. For this, we use the `data` endpoint and provide all the parameters we picked up throughout our exploration of the API:

In [45]:
# get NYC daily summaries data 
response = make_request(
    'data', 
    {
        'datasetid': 'GHCND',
        'stationid': central_park['id'],
        'locationid': nyc['id'],
        'startdate': '2018-10-01',
        'enddate': '2018-10-31',
        'datatypeid': ['TAVG', 'TMAX', 'TMIN'], # average, max, and min temperature
        'units': 'metric',
        'limit': 1000
    }
)
response.status_code

200

## Create a DataFrame
The Central Park station only has the daily minimum and maximum temperatures.

In [46]:
import pandas as pd

df = pd.DataFrame(response.json()['results'])
df.head()

,date,datatype,station,attributes,value
0,2018-10-01T00:00:00,TMAX,GHCND:USW00094728,",,W,2400",24.4
1,2018-10-01T00:00:00,TMIN,GHCND:USW00094728,",,W,2400",17.2
2,2018-10-02T00:00:00,TMAX,GHCND:USW00094728,",,W,2400",25.0
3,2018-10-02T00:00:00,TMIN,GHCND:USW00094728,",,W,2400",18.3
4,2018-10-03T00:00:00,TMAX,GHCND:USW00094728,",,W,2400",23.3


We didn't get `TAVG` because the station doesn't measure that:

In [47]:
df.datatype.unique()

array(['TMAX', 'TMIN'], dtype=object)

Despite showing up in the data as measuring it... Real-world data is dirty!

In [48]:
parameters = [
    'NY City Central Park', 
    {'locationid': nyc['id'], 'datatypeid': 'TAVG'}, 
    'stations'
    ]
if get_item(parameters[0], parameters[1], parameters[2]):
    print('Found!')

Found!


## Using a different station
Let's use LaGuardia airport instead. It contains `TAVG` (average daily temperature):

In [50]:
laguardia = get_item(
    'LaGuardia', 
    {'locationid': nyc['id']}, 
    'stations'
)
laguardia

{'elevation': 3.4,
 'mindate': '1939-10-07',
 'maxdate': '2021-08-27',
 'latitude': 40.77944,
 'name': 'LAGUARDIA AIRPORT, NY US',
 'datacoverage': 1,
 'id': 'GHCND:USW00014732',
 'elevationUnit': 'METERS',
 'longitude': -73.88035}

We make our request using the LaGuardia airport station this time.

In [52]:
# get NYC daily summaries data 

response = make_request(
    'data', 
    {
        'datasetid': 'GHCND',
        'stationid': laguardia['id'],
        'locationid': nyc['id'],
        'startdate': '2018-10-01',
        'enddate': '2018-10-31',
        'datatypeid': ['TAVG', 'TMAX', 'TMIN'], # temperature at time of observation, min, and max
        'units': 'metric',
        'limit': 1000
    }
)
response.status_code

200

The request was successful, so let's make a dataframe:

In [53]:
df = pd.DataFrame(response.json()['results'])
df.head()

,date,datatype,station,attributes,value
0,2018-10-01T00:00:00,TAVG,GHCND:USW00014732,"H,,S,",21.2
1,2018-10-01T00:00:00,TMAX,GHCND:USW00014732,",,W,2400",25.6
2,2018-10-01T00:00:00,TMIN,GHCND:USW00014732,",,W,2400",18.3
3,2018-10-02T00:00:00,TAVG,GHCND:USW00014732,"H,,S,",22.7
4,2018-10-02T00:00:00,TMAX,GHCND:USW00014732,",,W,2400",26.1


We should check that we got what we wanted: 31 entries for TAVG, TMAX, and TMIN (1 per day):

In [54]:
df.datatype.value_counts()

TMAX    31
TAVG    31
TMIN    31
Name: datatype, dtype: int64

Write the data to a CSV file for use in other notebooks.

In [43]:
df.to_csv('data/nyc_temperatures.csv', index=False)

<hr>
<div>
    <a href="./1-wide_vs_long.ipynb">
        <button>&#8592; Previous Notebook</button>
    </a>
    <a href="./3-cleaning_data.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<hr>